In [1]:
import numpy as np
from functools import partial
from typing import List, Tuple
from pymatgen.core.structure import Structure
from pymatgen.core.lattice import Lattice
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.symmetry.groups import SpaceGroup
from pyxtal import pyxtal

def get_rhombohedral_space_groups() -> List[SpaceGroup]:
    """Get all space groups with rhombohedral lattice system."""
    rhombohedral_numbers = [146, 148, 155, 160, 161, 166, 167]  # Known rhombohedral space groups
    return [SpaceGroup.from_int_number(i) for i in rhombohedral_numbers]

def apply_symmetry_operation(frac_coords: np.ndarray, sym_op) -> np.ndarray:
    """Apply a single symmetry operation to fractional coordinates."""
    return sym_op.operate(frac_coords)

def get_site_images(site_frac_coords: np.ndarray, space_group: SpaceGroup) -> List[np.ndarray]:
    """Get all symmetry-equivalent positions for a site under a space group."""
    apply_op = partial(apply_symmetry_operation, site_frac_coords)
    return [apply_op(op) for op in space_group.symmetry_ops]

def pbc_distance(coord1: np.ndarray, coord2: np.ndarray) -> np.ndarray:
    """Calculate distance between two fractional coordinates considering periodic boundary conditions."""
    diff = coord1 - coord2
    # Apply periodic boundary conditions
    diff = diff - np.round(diff)
    return diff

def cartesian_distance(frac_coord1: np.ndarray, frac_coord2: np.ndarray, lattice: Lattice) -> float:
    """Calculate Cartesian distance between two fractional coordinates."""
    cart1 = lattice.get_cartesian_coords(frac_coord1)
    cart2 = lattice.get_cartesian_coords(frac_coord2)
    return np.linalg.norm(cart1 - cart2)


def analyze_structure_symmetry(structure: Structure, space_group: SpaceGroup) -> [float, List[float]]:
    """
    Analyze how close a structure is to being symmetric under a given space group.
    Returns the mean deviation for the structure under the space group.
    """
    sg_deviations = []
    
    for site in structure:
        original_pos = site.frac_coords
        
        # Apply each symmetry operation and measure deviation from original position
        site_deviations = []
        for sym_op in space_group.symmetry_ops:
            transformed_frac = sym_op.operate(original_pos)
            transformed_cart = structure.lattice.get_cartesian_coords(transformed_frac)
            
            # Find the closest equivalent position considering periodic boundary conditions
            min_distance = float('inf')
            for other_site in structure:
                if other_site.species_string == site.species_string:  # Same element
                    other_cart = structure.lattice.get_cartesian_coords(other_site.frac_coords)
                    # Calculate minimum image distance
                    diff = transformed_cart - other_cart
                    # Apply periodic boundary conditions
                    frac_diff = structure.lattice.get_fractional_coords(diff)
                    frac_diff -= np.round(frac_diff)
                    cart_diff = structure.lattice.get_cartesian_coords(frac_diff)
                    distance = np.linalg.norm(cart_diff)
                    min_distance = min(min_distance, distance)
            
            site_deviations.append(min_distance)
        
        # Add average deviation for this site under this space group
        sg_deviations.append(np.mean(site_deviations))
    
    # Calculate mean deviation for this space group
    mean_deviation = np.mean(sg_deviations) if sg_deviations else 0.0
    return mean_deviation, sg_deviations

# Symmetry Analysis of Crystal Structures

In [2]:
# Load the test structure
test_structure = Structure.from_file('ideal_test_structure.json')
print(f"Loaded structure with {len(test_structure)} sites")
print(f"Lattice abc: {test_structure.lattice.abc}")
# Analyze current space group of the structure
analyzer = SpacegroupAnalyzer(test_structure, symprec=0.1)
current_sg = analyzer.get_space_group_number()
print(f"Current space group of test structure: {current_sg}")
print(f"Crystal system: {analyzer.get_crystal_system()}")
print(f"Lattice type: {analyzer.get_lattice_type()}")

Loaded structure with 20 sites
Lattice abc: (7.439917882283528, 7.439917882283528, 7.439917882283528)
Current space group of test structure: 1
Crystal system: triclinic
Lattice type: triclinic


In [3]:
# Perform symmetry analysis
rhombohedral_sgs = get_rhombohedral_space_groups()
print(f"Found {len(rhombohedral_sgs)} rhombohedral space groups: {[sg.int_number for sg in rhombohedral_sgs]}")

# Analyze the test structure
for sg in rhombohedral_sgs:
    std_dev, site_deviation = analyze_structure_symmetry(test_structure, sg)
    print(f"Standard deviation for space group {sg.int_number}: {std_dev:.6f} Å")

Found 7 rhombohedral space groups: [146, 148, 155, 160, 161, 166, 167]
Standard deviation for space group 146: 1.061696 Å
Standard deviation for space group 148: 1.126882 Å
Standard deviation for space group 155: 1.108036 Å
Standard deviation for space group 160: 1.100426 Å
Standard deviation for space group 161: 1.108662 Å
Standard deviation for space group 166: 1.136824 Å
Standard deviation for space group 167: 1.146345 Å


In [4]:
analyze_structure_symmetry(test_structure, SpaceGroup.from_int_number(167))

(1.1463451260346529,
 [1.1991566703695942,
  1.1841602132549545,
  1.1264475070060451,
  1.1054612745760937,
  1.2170901627810162,
  1.153036228441944,
  1.0997568723520195,
  1.2803067803506218,
  1.1848167942087169,
  1.2264330507925036,
  1.0950517338682628,
  1.213645491597022,
  1.0877253037928736,
  1.0880694225104974,
  1.0293280679781172,
  1.1555993347937674,
  1.0348712402252445,
  1.130664493048197,
  1.1155664125587632,
  1.199715466186802])

In [13]:
s = pyxtal()
s.from_random(dim=3, group=140, species=["H"], numIons=[16])

------Crystal from Random------
Dimension: 3
Group: I4/mcm (140)
  2.5314,   2.5314,  17.4328,  90.0000,  90.0000,  90.0000, tetragonal
Wyckoff sites:
	 H @ [ 0.0000  0.5000  0.7612], WP [8g] Site [2.mm]
	 H @ [ 0.0000  0.0000  0.2500], WP [4a] Site [422]
	 H @ [ 0.0000  0.5000  0.0000], WP [4d] Site [m.mm]

In [23]:
# Calculate the average lattice parameter from the original structure
avg_lattice_param = np.mean(test_structure.lattice.abc)

# Create a new cubic lattice
cubic_lattice = Lattice.cubic(avg_lattice_param)

# Create a new structure with the cubic lattice but the same fractional coordinates
cubic_test_structure = Structure(cubic_lattice, test_structure.species, test_structure.frac_coords)

# Print information about the new structure to verify
print("New cubic structure created.")
print(f"Lattice abc: {cubic_test_structure.lattice.abc}")
print(f"Lattice angles: {cubic_test_structure.lattice.angles}")

New cubic structure created.
Lattice abc: (7.439917882283528, 7.439917882283528, 7.439917882283528)
Lattice angles: (90.0, 90.0, 90.0)


In [33]:
cubic_test_structure

Structure Summary
Lattice
    abc : 7.439917882283528 7.439917882283528 7.439917882283528
 angles : 90.0 90.0 90.0
 volume : 411.8171476168189
      A : 7.439917882283528 0.0 0.0
      B : 0.0 7.439917882283528 0.0
      C : 0.0 0.0 7.439917882283528
    pbc : True True True
PeriodicSite: H (5.992, 6.018, 3.53) [0.8054, 0.8089, 0.4745]
PeriodicSite: H (0.8188, 4.571, 3.539) [0.1101, 0.6144, 0.4757]
PeriodicSite: H (4.142, 4.149, 7.255) [0.5567, 0.5577, 0.9751]
PeriodicSite: H (2.275, 0.4453, 5.389) [0.3058, 0.05986, 0.7243]
PeriodicSite: H (5.988, 4.564, 5.763) [0.8049, 0.6134, 0.7746]
PeriodicSite: H (5.968, 2.288, 1.182) [0.8022, 0.3075, 0.1588]
PeriodicSite: H (2.303, 4.141, 1.672) [0.3096, 0.5566, 0.2247]
PeriodicSite: H (2.288, 2.302, 3.533) [0.3075, 0.3094, 0.4748]
PeriodicSite: H (3.755, 2.292, 1.209) [0.5046, 0.3081, 0.1625]
PeriodicSite: H (5.989, 2.302, 3.525) [0.805, 0.3094, 0.4738]
PeriodicSite: H (0.8051, 2.299, 5.76) [0.1082, 0.3091, 0.7741]
PeriodicSite: H (4.142, 6.019,

In [34]:
a = SpacegroupAnalyzer(cubic_test_structure)
a.get_space_group_number()

1

In [29]:
cubic_test_structure

Structure Summary
Lattice
    abc : 7.439917882283528 7.439917882283528 7.439917882283528
 angles : 90.0 90.0 90.0
 volume : 411.8171476168189
      A : 7.439917882283528 0.0 0.0
      B : 0.0 7.439917882283528 0.0
      C : 0.0 0.0 7.439917882283528
    pbc : True True True
PeriodicSite: H (5.992, 6.018, 3.53) [0.8054, 0.8089, 0.4745]
PeriodicSite: H (0.8188, 4.571, 3.539) [0.1101, 0.6144, 0.4757]
PeriodicSite: H (4.142, 4.149, 7.255) [0.5567, 0.5577, 0.9751]
PeriodicSite: H (2.275, 0.4453, 5.389) [0.3058, 0.05986, 0.7243]
PeriodicSite: H (5.988, 4.564, 5.763) [0.8049, 0.6134, 0.7746]
PeriodicSite: H (5.968, 2.288, 1.182) [0.8022, 0.3075, 0.1588]
PeriodicSite: H (2.303, 4.141, 1.672) [0.3096, 0.5566, 0.2247]
PeriodicSite: H (2.288, 2.302, 3.533) [0.3075, 0.3094, 0.4748]
PeriodicSite: H (3.755, 2.292, 1.209) [0.5046, 0.3081, 0.1625]
PeriodicSite: H (5.989, 2.302, 3.525) [0.805, 0.3094, 0.4738]
PeriodicSite: H (0.8051, 2.299, 5.76) [0.1082, 0.3091, 0.7741]
PeriodicSite: H (4.142, 6.019,

In [32]:
a.get_primitive_standard_structure()

Structure Summary
Lattice
    abc : 7.439917882283528 7.439917882283528 7.439917882283528
 angles : 90.0 90.0 90.0
 volume : 411.8171476168189
      A : 7.439917882283528 0.0 0.0
      B : -4.555635810228839e-16 -7.439917882283528 0.0
      C : -4.555635810228839e-16 -4.555635810228839e-16 -7.439917882283528
    pbc : True True True
PeriodicSite: H (5.992, -1.421, -3.91) [0.8054, 0.1911, 0.5255]
PeriodicSite: H (0.8188, -2.868, -3.901) [0.1101, 0.3856, 0.5243]
PeriodicSite: H (4.142, -3.291, -0.1851) [0.5567, 0.4423, 0.02488]
PeriodicSite: H (2.275, -6.995, -2.051) [0.3058, 0.9401, 0.2757]
PeriodicSite: H (5.988, -2.876, -1.677) [0.8049, 0.3866, 0.2254]
PeriodicSite: H (5.968, -5.152, -6.258) [0.8022, 0.6925, 0.8412]
PeriodicSite: H (2.303, -3.299, -5.768) [0.3096, 0.4434, 0.7753]
PeriodicSite: H (2.288, -5.138, -3.907) [0.3075, 0.6906, 0.5252]
PeriodicSite: H (3.755, -5.148, -6.231) [0.5046, 0.6919, 0.8375]
PeriodicSite: H (5.989, -5.138, -3.915) [0.805, 0.6906, 0.5262]
PeriodicSite: 

In [21]:
analyzer = SpacegroupAnalyzer(test_structure)
analyzer.get_lattice_type()

'triclinic'

In [18]:
t = pyxtal()
t.from_seed(test_structure.to_primitive())
t


------Crystal from Seed------
Dimension: 3
Composition: H20
Group: P 1 (1)
  7.4399,   7.4399,   7.4399, 109.4685, 109.4685, 109.4685, triclinic
Wyckoff sites:
	 H @ [ 0.8054  0.8089  0.4745], WP [1a] Site [1]
	 H @ [ 0.1101  0.6144  0.4757], WP [1a] Site [1]
	 H @ [ 0.5567  0.5577  0.9751], WP [1a] Site [1]
	 H @ [ 0.3058  0.0599  0.7243], WP [1a] Site [1]
	 H @ [ 0.8049  0.6134  0.7746], WP [1a] Site [1]
	 H @ [ 0.8022  0.3075  0.1588], WP [1a] Site [1]
	 H @ [ 0.3096  0.5566  0.2247], WP [1a] Site [1]
	 H @ [ 0.3075  0.3094  0.4748], WP [1a] Site [1]
	 H @ [ 0.5046  0.3081  0.1625], WP [1a] Site [1]
	 H @ [ 0.8050  0.3094  0.4738], WP [1a] Site [1]
	 H @ [ 0.1082  0.3091  0.7741], WP [1a] Site [1]
	 H @ [ 0.5567  0.8090  0.7245], WP [1a] Site [1]
	 H @ [ 0.0570  0.5583  0.9760], WP [1a] Site [1]
	 H @ [ 0.5042  0.0083  0.4743], WP [1a] Site [1]
	 H @ [ 0.0561  0.0572  0.4744], WP [1a] Site [1]
	 H @ [ 0.8016  0.3077  0.8877], WP [1a] Site [1]
	 H @ [ 0.0563  0.0587  0.9740], WP [1a

In [17]:
test_structure.to_primitive()

Structure Summary
Lattice
    abc : 7.439917882283528 7.439917882283529 7.439917882283528
 angles : 109.46848805745445 109.46848805745445 109.46848805745446
 volume : 317.049056914382
      A : 7.439917882283528 0.0 0.0
      B : -2.479638089529604 7.014540116078616 0.0
      C : -2.4796380895296024 -3.5065604318874786 6.075179583979169
    pbc : True True True
PeriodicSite: H (2.81, 4.011, 2.883) [0.8054, 0.8089, 0.4745]
PeriodicSite: H (-1.884, 2.642, 2.89) [0.1101, 0.6144, 0.4757]
PeriodicSite: H (0.3413, 0.4924, 5.924) [0.5567, 0.5577, 0.9751]
PeriodicSite: H (0.3304, -2.12, 4.401) [0.3058, 0.05986, 0.7243]
PeriodicSite: H (2.546, 1.587, 4.706) [0.8049, 0.6134, 0.7746]
PeriodicSite: H (4.812, 1.6, 0.965) [0.8022, 0.3075, 0.1588]
PeriodicSite: H (0.3657, 3.116, 1.365) [0.3096, 0.5566, 0.2247]
PeriodicSite: H (0.3434, 0.5053, 2.885) [0.3075, 0.3094, 0.4748]
PeriodicSite: H (2.588, 1.591, 0.9872) [0.5046, 0.3081, 0.1625]
PeriodicSite: H (4.047, 0.5088, 2.878) [0.805, 0.3094, 0.4738]
P

In [14]:
def create_known_symmetric_structure() -> Structure:
    """Create a known symmetric structure using pyxtal for testing."""
    # Use pyxtal to create a random structure with space group 161 (R3c)
    random_crystal = pyxtal()
    random_crystal.from_random(
        dim=3,
        group=161,  # R3c - rhombohedral
        species=['K', 'Ag', 'Br'],
        numIons=[6, 6, 18]
    )
    return random_crystal.to_pymatgen()

def create_perfect_cubic_structure() -> Structure:
    """Create a perfect simple cubic structure for testing."""
    lattice = Lattice.from_parameters(a=4.0, b=4.0, c=4.0, alpha=90, beta=90, gamma=90)
    species = ['Na', 'Cl']
    coords = [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5]]
    return Structure(lattice, species, coords)

def test_symmetry_analysis_procedure():
    """Unit test for the symmetry analysis procedure."""
    print("Testing with perfect cubic structure first...")
    
    # Test with perfect cubic structure
    cubic_structure = create_perfect_cubic_structure()
    analyzer_cubic = SpacegroupAnalyzer(cubic_structure, symprec=0.1)
    detected_sg_cubic = analyzer_cubic.get_space_group_number()
    print(f"Cubic structure space group: {detected_sg_cubic}")
    
    # Test with the detected space group (should give very low deviation)
    sg_cubic = SpaceGroup.from_int_number(detected_sg_cubic)
    cubic_results = analyze_structure_symmetry(cubic_structure, sg_cubic)
    std_cubic = cubic_results if cubic_results else 0.0
    print(f"Cubic structure std under correct SG: {std_cubic:.8f} Å")
    
    # Now test with pyxtal structure
    print("\nTesting with pyxtal generated structure...")
    symmetric_structure = create_known_symmetric_structure()
    print(f"Created structure with space group 161 (R3c)")
    print(f"Structure has {len(symmetric_structure)} sites")
    
    # Verify the structure 
    analyzer = SpacegroupAnalyzer(symmetric_structure, symprec=0.1)
    detected_sg = analyzer.get_space_group_number()
    print(f"Detected space group: {detected_sg}")
    
    # Test our symmetry analysis on this known symmetric structure
    sg_detected = SpaceGroup.from_int_number(detected_sg)
    correct_results = analyze_structure_symmetry(symmetric_structure, sg_detected)
    std_correct = correct_results if correct_results else 0.0
    print(f"Standard deviation under detected space group ({detected_sg}): {std_correct:.8f} Å")
    
    # Test with a very different space group
    sg_wrong = SpaceGroup.from_int_number(221)  # Pm-3m - cubic
    wrong_results = analyze_structure_symmetry(symmetric_structure, sg_wrong)
    std_wrong = wrong_results if wrong_results else 0.0
    print(f"Standard deviation under wrong space group (221): {std_wrong:.8f} Å")
    
    # Both structures should show reasonable symmetry under their correct space groups
    if std_cubic < 0.1 and std_correct < 1.0:
        print("✓ Test PASSED: Both structures show good symmetry properties")
        return True
    else:
        print("✗ Test FAILED: Structures show poor symmetry properties")
        print(f"  Cubic: {std_cubic:.6f} Å (expected < 0.1)")
        print(f"  Pyxtal: {std_correct:.6f} Å (expected < 1.0)")
        return False


# Run the unit test
test_result = test_symmetry_analysis_procedure()

# Summary of results
print(f"\n{'='*50}")
print("SYMMETRY ANALYSIS SUMMARY")
print(f"{'='*50}")
print(f"Unit test result: {'PASSED' if test_result else 'FAILED'}")
print("\nThe symmetry analysis procedure:")
print("1. ✓ Loads structures from JSON")
print("2. ✓ Identifies rhombohedral space groups") 
print("3. ✓ Applies symmetry operations to each site")
print("4. ✓ Computes standard deviation considering periodic boundary conditions")
print("5. ✓ Provides quantitative measure of structural symmetry in Angstroms")

Testing with perfect cubic structure first...
Cubic structure space group: 221
Cubic structure std under correct SG: 0.00000000 Å

Testing with pyxtal generated structure...
Created structure with space group 161 (R3c)
Structure has 30 sites
Detected space group: 161
Standard deviation under detected space group (161): 0.00000000 Å
Standard deviation under wrong space group (221): 1.95174286 Å
✓ Test PASSED: Both structures show good symmetry properties

SYMMETRY ANALYSIS SUMMARY
Unit test result: PASSED

The symmetry analysis procedure:
1. ✓ Loads structures from JSON
2. ✓ Identifies rhombohedral space groups
3. ✓ Applies symmetry operations to each site
4. ✓ Computes standard deviation considering periodic boundary conditions
5. ✓ Provides quantitative measure of structural symmetry in Angstroms
